<h3>Importing data

In [1]:
import pandas as pd
data_train=pd.read_csv("train.csv")
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 13 columns):
X        450 non-null int64
Y        450 non-null int64
month    450 non-null object
day      450 non-null object
FFMC     450 non-null float64
DMC      450 non-null float64
DC       450 non-null float64
ISI      450 non-null float64
temp     450 non-null float64
RH       450 non-null int64
wind     450 non-null float64
rain     450 non-null float64
area     450 non-null float64
dtypes: float64(8), int64(3), object(2)
memory usage: 45.8+ KB


In [2]:
data_train_update=pd.DataFrame({"X":[],"Y":[],"month":[],"day":[],"FFMC":[],"DMC":[],"DC":[],"ISI":[],"temp":[],"RH":[],"wind":[],"rain":[],"area":[]})
for key in data_train:
    if key != "month" and key != "day":
        data_train_update[key]=data_train[key]
mon_dict={"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12}
day_dict={"mon":1,"tue":2,"wed":3,"thu":4,"fri":5,"sat":6,"sun":7}
for i in range(0,450):
    data_train_update["month"][i]=mon_dict[data_train["month"][i]]
    data_train_update["day"][i]=day_dict[data_train["day"][i]]

/home/lokesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/lokesh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [3]:
data_train_update_outrm=data_train_update[data_train_update.area<=250]
data_train_update_outrm=data_train_update_outrm[data_train_update_outrm.rain!=6.40000]

In [4]:
import math
data_train_update_outrm["area"]=data_train_update_outrm["area"].apply(lambda x:0.69 if x==0 else math.log(x)/math.log(2))

In [5]:
data_train_update_outrm.head()

,DC,DMC,FFMC,ISI,RH,X,Y,area,day,month,rain,temp,wind
0,94.3,26.2,86.2,5.1,51,7,5,0.69,5.0,3.0,0.0,8.2,6.7
1,669.1,35.4,90.6,6.7,33,7,4,0.69,2.0,10.0,0.0,18.0,0.9
2,77.5,33.3,91.7,9.0,97,8,6,0.69,5.0,3.0,0.2,8.3,4.0
3,488.0,85.3,92.3,14.7,29,8,6,0.69,7.0,8.0,0.0,22.2,5.4
4,495.6,88.9,92.3,8.5,27,8,6,0.69,1.0,8.0,0.0,24.1,3.1


In [6]:
from sklearn.model_selection import train_test_split
labels=pd.DataFrame({"area":[]})
labels["area"]=data_train_update_outrm["area"]
features=data_train_update_outrm.drop("area",axis="columns")
features_train,features_test,labels_train,labels_test=train_test_split(features,labels,test_size=0.33)

<h2>Applying SVM Regressor

In [7]:
from sklearn.metrics import r2_score
from sklearn.svm import SVR
svmr=SVR(kernel='rbf', C=100.0, gamma='auto')
svmr.fit(features_train,labels_train)
pred=svmr.predict(features_test)

/home/lokesh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
from sklearn.metrics import mean_squared_error
mean_squared_error(labels_test,pred)

2.6455617104059925

In [9]:
data_test=pd.read_csv('test.csv')
ans=pd.DataFrame()
ans['Id']=data_test['Id']
data_test.drop('Id',axis='columns',inplace=True)
data_test.replace({"day":day_dict},inplace=True)
data_test.replace({"month":mon_dict},inplace=True)
svmr.fit(features,labels)

/home/lokesh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=100.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [10]:
pred=svmr.predict(data_test)
pred=2**pred
ans['area']=pred
ans.to_csv('sampleSubmission.csv')

<h2>Applying XGBoost Regressor

In [26]:
import xgboost as xgb
#eval_set=[(test_df,labels_test)]
reg = xgb.XGBRegressor(max_depth=15, n_estimators=350, learning_rate=0.1)
reg.fit(features_train,labels_train,eval_metric="error",verbose=True)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=15, min_child_weight=1, missing=None, n_estimators=350,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [27]:
pred=reg.predict(features_test)
from sklearn.metrics import mean_squared_error
mean_squared_error(labels_test, pred)

3.5137912264316884

<h2>Applying adaboost

In [15]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [18]:
from sklearn.metrics import mean_squared_error
reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),n_estimators=50,loss='exponential',learning_rate=0.01)
reg.fit(features,labels)
pred = reg.predict(data_test)
pred

/home/lokesh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([2.54485562, 1.89626305, 2.31283501, 2.31283501, 2.42760617,
       2.42760617, 2.54485562, 2.54485562, 2.54485562, 2.54485562,
       2.43028527, 2.43028527, 2.42760617, 2.31283501, 2.31283501,
       2.54485562, 2.43028527, 2.54485562, 2.42760617, 2.23440852,
       2.54485562, 1.47824742, 2.43028527, 2.23440852, 2.42760617,
       2.43028527, 2.31283501, 2.42760617, 2.43028527, 2.54485562,
       2.31283501, 2.54485562, 2.23440852, 2.43028527, 2.42760617,
       2.31283501, 2.42760617, 2.31283501, 2.42760617, 2.42760617,
       2.43028527, 2.54485562, 2.42760617, 2.23440852, 2.54485562,
       1.47824742, 2.42760617, 2.43028527, 2.42760617, 2.31283501,
       2.42760617, 2.43028527, 1.46703534, 1.46703534, 2.42760617,
       2.43028527, 2.42760617, 2.43028527, 2.42760617, 2.43028527,
       2.43028527, 2.54485562, 2.54485562, 2.54485562, 2.43028527,
       2.54485562, 2.54485562])

In [20]:
pred=reg.predict(features_test)
from sklearn.metrics import mean_squared_error
mean_squared_error(labels_test, pred)

2.058921884658765

In [21]:
from sklearn.metrics import r2_score
from sklearn.svm import SVR
svmr=SVR(kernel='rbf', C=100, gamma=0.1)
svmr.fit(features,labels)
pred = svmr.predict(data_test)
pred=2**pred

/home/lokesh/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
